In [3]:
"""
RNN은 기존의 뉴럴넷과 다른점이 있다면, 이전의 output이 다음의 cell로 연결된다는 것.
텐서플로우를 사용하면 쉽게 구현가능
"""

'\nRNN은 기존의 뉴럴넷과 다른점이 있다면, 이전의 output이 다음의 cell로 연결된다는 것.\n텐서플로우를 사용하면 쉽게 구현가능\n'

In [16]:
import tensorflow as tf
import numpy as np
from tensorflow.contrib import rnn
import pprint
pp = pprint.PrettyPrinter(indent=4)
sess = tf.InteractiveSession()

In [17]:
# One hot encoding
h = [1, 0, 0, 0]
e = [0, 1, 0, 0]
l = [0, 0, 1, 0]
o = [0, 0, 0, 1]
# shape=(1,1,4) # input shape [[[1,0,0,0]]]
hidden_size=2 # hidden size를 무엇으로 정했는지에따라 출력이 결정된다.
# input data의 dimension이 뭐든 상관없이 출력의 값을 2개로 주겠다..라는 것
# -> output shape=(1,1,2)   [[[x,x]]]
# 이런건 cell을 만들때 정의하는 것임

In [18]:
"""
step 1
cell을 만들어야 한다. 뭐 어떤 형태의 cell을 만들것인가? 뭐  basic, lstm, gru
cell을 만들때 가장 중요한 것은 
cell에서 나가는 output의 size가 얼마일까(hidden_size 혹은 num_units라고 함)
"""
# cell = tf.contrib.rnn.BasicRNNCell(num_units=hidden_size)

"""
step 2
그럼 이 cell을 만든 것을 구동을 시켜서 입력을 주고 출력을 뽑아야하는데, 
이것을 보통 드라이버라고 부를 수도 있고, 다른 형태의 함수; wrapper라고 볼수 있는데 보통
tf.nn.dynamic_rnn을 쓰게 된다.
여기에 만든 cell을 넘겨주고 우리가 원하는 input을 넘겨주게 된다.
그럼 두가지 출력을 내는데,
하나는 output, 또하나는 마지막 state의 값을 준다.
state는 우리가 잘 사용할 일이 없다
"""
# outputs, _states = tf.nn.dynamic_rnn(cell, x_data, dtype=tf.float32)

'\nstep 2\n그럼 이 cell을 만든 것을 구동을 시켜서 입력을 주고 출력을 뽑아야하는데, \n이것을 보통 드라이버라고 부를 수도 있고, 다른 형태의 함수; wrapper라고 볼수 있는데 보통\ntf.nn.dynamic_rnn을 쓰게 된다.\n여기에 만든 cell을 넘겨주고 우리가 원하는 input을 넘겨주게 된다.\n그럼 두가지 출력을 내는데,\n하나는 output, 또하나는 마지막 state의 값을 준다.\nstate는 우리가 잘 사용할 일이 없다\n'

In [28]:
"""
cell생성 부분과 cell을 통해 학습하고 구동하는 부분을 나누어 줌으로써 
원하는 형태의 cell을 맘대로 바꿔줄 수가 있다.
"""

# cell = tf.contrib.rnn.BasicRNNCell(num_units=hidden_size)
with tf.variable_scope('one_cell') as scope:
    # One cell RNN input_dim (4) -> output_dim (2)
    hidden_size = 2
    cell = tf.contrib.rnn.BasicLSTMCell(num_units=hidden_size)

    x_data = np.array([[[1,0,0,0]]], dtype=np.float32)
    pp.pprint(x_data)
    outputs, _states = tf.nn.dynamic_rnn(cell, x_data, dtype=tf.float32)

    sess.run(tf.global_variables_initializer())
    pp.pprint(outputs.eval())

array([[[ 1.,  0.,  0.,  0.]]], dtype=float32)


ValueError: Variable one_cell/rnn/basic_lstm_cell/weights already exists, disallowed. Did you mean to set reuse=True in VarScope? Originally defined at:

  File "/Users/junkyu/.pyenv/versions/3.5.2/envs/tensorflow-env-3.5.2/lib/python3.5/site-packages/tensorflow/contrib/rnn/python/ops/core_rnn_cell_impl.py", line 747, in _linear
    "weights", [total_arg_size, output_size], dtype=dtype)
  File "/Users/junkyu/.pyenv/versions/3.5.2/envs/tensorflow-env-3.5.2/lib/python3.5/site-packages/tensorflow/contrib/rnn/python/ops/core_rnn_cell_impl.py", line 179, in __call__
    concat = _linear([inputs, h], 4 * self._num_units, True, scope=scope)
  File "<ipython-input-20-afa2c6d20c79>", line 13, in <module>
    outputs, _states = tf.nn.dynamic_rnn(cell, x_data, dtype=tf.float32)


In [25]:
"""
shape=(1,5,4)
sequence_length: series data를 몇개를 받을 것인가에 대한 값; 한번에 cell을 몇번 펼칠 것인가
이건 입력데이터를 줄때 그 입력 어레이의 형태에 의해 알아서 결정된다.
"""
# One hot encoding
h = [1, 0, 0, 0]
e = [0, 1, 0, 0]
l = [0, 0, 1, 0]
o = [0, 0, 0, 1]

with tf.variable_scope('two_sequences') as scope:
    # One cell RNN input_dim (4) -> output_dim (2). sequence: 5
    hidden_size = 2
    cell = tf.contrib.rnn.BasicRNNCell(num_units=hidden_size)
    print(cell.output_size, cell.state_size)
    
    # 아래와 같이 sequence size를 만들어준다. seq = 5
    x_data = np.array([[h, e, l, l, o]], dtype=np.float32)
    print(x_data.shape)
    pp.pprint(x_data)
    outputs, _states = tf.nn.dynamic_rnn(cell, x_data, dtype=tf.float32)
    sess.run(tf.global_variables_initializer())
    pp.pprint(outputs.eval())
    # outputs들은 각 cell의 weight들일 것임.

2 2
(1, 5, 4)
array([[[ 1.,  0.,  0.,  0.],
        [ 0.,  1.,  0.,  0.],
        [ 0.,  0.,  1.,  0.],
        [ 0.,  0.,  1.,  0.],
        [ 0.,  0.,  0.,  1.]]], dtype=float32)
array([[[ 0.15969434, -0.33226603],
        [ 0.22477764,  0.26550105],
        [ 0.09447759, -0.33812353],
        [-0.45583069, -0.68295962],
        [-0.43204474,  0.28075731]]], dtype=float32)


In [26]:
# batching input
"""
batch size를 통해 여러데이터를 줄 수 있음.
"""

with tf.variable_scope('3_batches') as scope:
    # One cell RNN input_dim (4) -> output_dim (2). sequence: 5, batch 3
    # 3 batches 'hello', 'eolll', 'lleel'
    x_data = np.array([[h, e, l, l, o],
                       [e, o, l, l, l],
                       [l, l, e, e, l]], dtype=np.float32)

    pp.pprint(x_data)
    
    hidden_size = 2
    cell = rnn.BasicLSTMCell(num_units=hidden_size, state_is_tuple=True)
    outputs, _states = tf.nn.dynamic_rnn(
        cell, x_data, dtype=tf.float32)
    sess.run(tf.global_variables_initializer())
    pp.pprint(outputs.eval())

array([[[ 1.,  0.,  0.,  0.],
        [ 0.,  1.,  0.,  0.],
        [ 0.,  0.,  1.,  0.],
        [ 0.,  0.,  1.,  0.],
        [ 0.,  0.,  0.,  1.]],

       [[ 0.,  1.,  0.,  0.],
        [ 0.,  0.,  0.,  1.],
        [ 0.,  0.,  1.,  0.],
        [ 0.,  0.,  1.,  0.],
        [ 0.,  0.,  1.,  0.]],

       [[ 0.,  0.,  1.,  0.],
        [ 0.,  0.,  1.,  0.],
        [ 0.,  1.,  0.,  0.],
        [ 0.,  1.,  0.,  0.],
        [ 0.,  0.,  1.,  0.]]], dtype=float32)
array([[[-0.10906062,  0.05750896],
        [ 0.00738194, -0.085712  ],
        [ 0.15759379,  0.08211433],
        [ 0.28534856,  0.18914004],
        [ 0.41674593,  0.08413108]],

       [[ 0.09222777, -0.10889792],
        [ 0.18850979, -0.04409115],
        [ 0.27115873,  0.10533011],
        [ 0.36266097,  0.2067491 ],
        [ 0.4219299 ,  0.26672372]],

       [[ 0.15875447,  0.12790295],
        [ 0.28807002,  0.21608226],
        [ 0.35791621,  0.0764155 ],
        [ 0.37852645, -0.04007913],
        [ 0.3931796 ,

In [27]:
with tf.variable_scope('3_batches_dynamic_length') as scope:
    # One cell RNN input_dim (4) -> output_dim (5). sequence: 5, batch 3
    # 3 batches 'hello', 'eolll', 'lleel'
    x_data = np.array([[h, e, l, l, o],
                       [e, o, l, l, l],
                       [l, l, e, e, l]], dtype=np.float32)

    pp.pprint(x_data)
    
    hidden_size = 2
    cell = rnn.BasicLSTMCell(num_units=hidden_size, state_is_tuple=True)
    outputs, _states = tf.nn.dynamic_rnn(
        cell, x_data, sequence_length=[5,3,4], dtype=tf.float32)
    sess.run(tf.global_variables_initializer())
    pp.pprint(outputs.eval())

array([[[ 1.,  0.,  0.,  0.],
        [ 0.,  1.,  0.,  0.],
        [ 0.,  0.,  1.,  0.],
        [ 0.,  0.,  1.,  0.],
        [ 0.,  0.,  0.,  1.]],

       [[ 0.,  1.,  0.,  0.],
        [ 0.,  0.,  0.,  1.],
        [ 0.,  0.,  1.,  0.],
        [ 0.,  0.,  1.,  0.],
        [ 0.,  0.,  1.,  0.]],

       [[ 0.,  0.,  1.,  0.],
        [ 0.,  0.,  1.,  0.],
        [ 0.,  1.,  0.,  0.],
        [ 0.,  1.,  0.,  0.],
        [ 0.,  0.,  1.,  0.]]], dtype=float32)
array([[[-0.00645624,  0.18257737],
        [ 0.091427  ,  0.07146088],
        [ 0.00766226,  0.10999069],
        [-0.10878152,  0.12820807],
        [-0.03603111, -0.00460226]],

       [[ 0.08416969, -0.07597371],
        [ 0.11446248, -0.13593888],
        [-0.03475482, -0.10425442],
        [ 0.        ,  0.        ],
        [ 0.        ,  0.        ]],

       [[-0.13012083,  0.03296331],
        [-0.22653149,  0.04358187],
        [-0.01091214, -0.0578985 ],
        [ 0.07281873, -0.13319963],
        [ 0.        ,

In [31]:
with tf.variable_scope('initial_state') as scope:
    batch_size = 3
    x_data = np.array([[h, e, l, l, o],
                      [e, o, l, l, l],
                      [l, l, e, e, l]], dtype=np.float32)
    pp.pprint(x_data)
    
    # One cell RNN input_dim(4) -> output_dim (5). sequence: 5, batch: 3
    hidden_size=2
    cell = rnn.BasicLSTMCell(num_units=hidden_size, state_is_tuple=True)
    initial_state = cell.zero_state(batch_size, tf.float32)
    outputs, _states = tf.nn.dynamic_rnn(cell, x_data,
                                        initial_state=initial_state, dtype=tf.float32)
    sess.run(tf.global_variables_initializer())
    pp.pprint(outputs.eval())

array([[[ 1.,  0.,  0.,  0.],
        [ 0.,  1.,  0.,  0.],
        [ 0.,  0.,  1.,  0.],
        [ 0.,  0.,  1.,  0.],
        [ 0.,  0.,  0.,  1.]],

       [[ 0.,  1.,  0.,  0.],
        [ 0.,  0.,  0.,  1.],
        [ 0.,  0.,  1.,  0.],
        [ 0.,  0.,  1.,  0.],
        [ 0.,  0.,  1.,  0.]],

       [[ 0.,  0.,  1.,  0.],
        [ 0.,  0.,  1.,  0.],
        [ 0.,  1.,  0.,  0.],
        [ 0.,  1.,  0.,  0.],
        [ 0.,  0.,  1.,  0.]]], dtype=float32)
array([[[-0.07022887,  0.07325557],
        [-0.10870773,  0.04461332],
        [-0.21258216,  0.06556352],
        [-0.24651808,  0.0781047 ],
        [-0.22741725,  0.03849515]],

       [[-0.07597632,  0.00397045],
        [-0.12754203, -0.02652754],
        [-0.19907251,  0.02990212],
        [-0.23771062,  0.05684341],
        [-0.26317525,  0.07246798]],

       [[-0.11238696,  0.04529959],
        [-0.18382685,  0.06915338],
        [-0.16327067,  0.05812208],
        [-0.18752436,  0.04390874],
        [-0.28457615,

In [33]:
# Create input data
batch_size=3
sequence_length=5
input_dim=3

x_data = np.arange(45, dtype=np.float32).reshape(batch_size, sequence_length, input_dim)
pp.pprint(x_data)

array([[[  0.,   1.,   2.],
        [  3.,   4.,   5.],
        [  6.,   7.,   8.],
        [  9.,  10.,  11.],
        [ 12.,  13.,  14.]],

       [[ 15.,  16.,  17.],
        [ 18.,  19.,  20.],
        [ 21.,  22.,  23.],
        [ 24.,  25.,  26.],
        [ 27.,  28.,  29.]],

       [[ 30.,  31.,  32.],
        [ 33.,  34.,  35.],
        [ 36.,  37.,  38.],
        [ 39.,  40.,  41.],
        [ 42.,  43.,  44.]]], dtype=float32)


In [34]:
with tf.variable_scope('generated_data') as scope:
    # One cell RNN input_dum (3) -> output_dim (5). sequence: 5, batch: 3
    cell = rnn.BasicLSTMCell(num_units=5, state_is_tuple=True)
    initial_state = cell.zero_state(batch_size, tf.float32)
    outputs, _states = tf.nn.dynamic_rnn(cell, x_data,
                                        initial_state=initial_state, dtype=tf.float32)
    sess.run(tf.global_variables_initializer())
    pp.pprint(outputs.eval())

array([[[  6.15068600e-02,  -7.43666217e-02,   4.65881824e-02,
           5.86086847e-02,  -9.43710878e-02],
        [  1.02229059e-01,  -8.10933486e-02,   8.56113359e-02,
          -5.01152277e-02,  -7.56732225e-02],
        [  8.67768228e-02,  -4.30066176e-02,   7.35513419e-02,
          -6.15403913e-02,  -1.32910563e-02],
        [  6.05235547e-02,  -1.82999838e-02,   4.87128533e-02,
          -3.60763706e-02,  -1.38329586e-03],
        [  3.95366997e-02,  -6.95752772e-03,   2.82309819e-02,
          -1.92505363e-02,  -1.86250967e-04]],

       [[  1.80041985e-04,  -2.51258304e-03,   9.76169249e-04,
          -9.80630610e-03,  -2.75549337e-05],
        [  1.50887732e-04,  -8.76367674e-04,   8.18992732e-04,
          -4.97637223e-03,  -4.11127485e-06],
        [  1.02296261e-04,  -3.02609318e-04,   5.18253539e-04,
          -2.48353858e-03,  -6.05849834e-07],
        [  6.56756529e-05,  -1.03831517e-04,   2.95369449e-04,
          -1.22612470e-03,  -8.90204319e-08],
        [  4.1445

In [37]:
""" Cost: sequence_loss 
그 후에 output이 얼마나 좋은가를 loss를 통해 계산을 해야함. 그래야 학습이 가능하지
나온 값의 sequence를 다 알고 있으니 각각을 뭐 cross_entropy같은 걸로 계산해서 평균을 낸다던지 해서
할 수 있긴하다. 근데 복잡해질 수 있으니
tensorflow에서 sequence_loss라는 것이 있음.


"""

# [batch_size, sequence_length]
y_data = tf.constant([[1, 1, 1]])

# [batch_size, sequence_length, emb_dim ]

# onehot으로 나타냄(softmax랑 상당히 유사)
prediction = tf.constant([[[0.2, 0.7], [0.6, 0.2], [0.2, 0.9]]], dtype=tf.float32) 

# [batch_size * sequence_length]
weights = tf.constant([[1, 1, 1]], dtype=tf.float32)

"""
logits: 우리의 예측
targets: true data
weights: 각각의 자리를 얼마나 중요하게 생각하느냐.. 그냥 모두다 1이라 생각
"""
sequence_loss = tf.contrib.seq2seq.sequence_loss(logits=prediction, targets=y_data, weights=weights)
sess.run(tf.global_variables_initializer())
print('Loss: ', sequence_loss.eval())

Loss:  0.596759


In [38]:
# [batch_size, sequence_length]
y_data = tf.constant([[1, 1, 1]])

# [batch_size, sequence_length, emb_dim]
prediction1 = tf.constant([[[0.3, 0.7], [0.3, 0.7], [0.3, 0.7]]],
                         dtype=tf.float32)
prediction2 = tf.constant([[[0.1, 0.9], [0.1, 0.9], [0.1, 0.9]]],
                         dtype=tf.float32) # prediction1보다 강하게 1로 예측

# [batch_size * sequence_length]
weights = tf.constant([[1, 1, 1]], dtype=tf.float32)

sequence_loss1 = tf.contrib.seq2seq.sequence_loss(prediction1, y_data, weights)
sequence_loss2 = tf.contrib.seq2seq.sequence_loss(prediction2, y_data, weights)

sess.run(tf.global_variables_initializer())
print("Loss1: ", sequence_loss1.eval(),
     "Loss2: ", sequence_loss2.eval())
"""
원하는 값이 실제에 가까워질수록 loss가 1에 비해 낮아진다.
"""

Loss1:  0.513015 Loss2:  0.371101
